In [8]:
#Import libraires and read data file

import boto3
import json
import pandas as pd
import math

fileInput="movies_metadata.csv"
dfMoviesMetadata = pd.read_csv(fileInput, keep_default_na=False)

In [9]:
dfMoviesMetadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415
1,False,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [10]:
# Load dimensions as keys
dfMoviesMetadataKeys = dfMoviesMetadata[['id', 'revenue', 'vote_average']]
dfMoviesMetadataKeys.head()

,id,revenue,vote_average
0,862,373554033,7.7
1,8844,262797249,6.9
2,15602,0,6.5
3,31357,81452156,6.1
4,11862,76578911,5.7


In [11]:
dfMoviesMetadataInfo = dfMoviesMetadata[['adult', 'belongs_to_collection', 'budget',
                                        'genres', 'homepage', 'imdb_id', 'original_language',
                                        'original_title','overview', 'popularity',
                                        'production_companies', 'production_countries',
                                        'revenue', 'runtime', 'spoken_languages',
                                        'release_date','title', 'video', 'vote_count']]

dfMoviesMetadataInfo.head()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,release_date,title,video,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",1995-10-30,Toy Story,False,5415
1,False,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",1995-12-15,Jumanji,False,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",1995-12-22,Grumpier Old Men,False,92
3,False,,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",1995-12-22,Waiting to Exhale,False,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",1995-02-10,Father of the Bride Part II,False,173


In [12]:
dfMoviesMetadataKeys.dtypes

id              object
revenue         object
vote_average    object
dtype: object

In [ ]:
# Drop 'dirty' records    
dfMoviesMetadataInfo = dfMoviesMetadataInfo.drop([19730])
dfMoviesMetadataKeys = dfMoviesMetadataKeys.drop([19730])
dfMoviesMetadataInfo = dfMoviesMetadataInfo.drop([29503])
dfMoviesMetadataKeys = dfMoviesMetadataKeys.drop([29503])
dfMoviesMetadataInfo = dfMoviesMetadataInfo.drop([35587])
dfMoviesMetadataKeys = dfMoviesMetadataKeys.drop([35587])

In [59]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-1', endpoint_url="https://dynamodb.us-west-1.amazonaws.com")
table = dynamodb.Table('movies_metadata')
index=0

# Convert id to numeric
dfMoviesMetadataKeys['id']=pd.to_numeric(dfMoviesMetadataKeys['id'])
dfMoviesMetadataKeys['revenue']=pd.to_numeric(dfMoviesMetadataKeys['revenue'])
dfMoviesMetadataKeys['vote_average']=pd.to_numeric(dfMoviesMetadataKeys['vote_average'])

for index in dfMoviesMetadataKeys.index.tolist():
    movieId = int(dfMoviesMetadataKeys.loc[index, 'id'])
    
    if math.isnan(dfMoviesMetadataKeys.loc[index, 'revenue']):
        revenue=0
    else:
        revenue = int(dfMoviesMetadataKeys.loc[index, 'revenue'])
        
    if math.isnan(dfMoviesMetadataKeys.loc[index, 'vote_average']):
        vote_average=0
    else:
        revenue = int(dfMoviesMetadataKeys.loc[index, 'vote_average'])

    info = dfMoviesMetadataInfo.loc[index].to_json()
    
    table.put_item(
        Item = {
        'id': movieId,
        'revenue': revenue,
        'vote_average': vote_average,
        'info': info
        }
    )

TypeError: Unsupported type "<class 'pandas.core.series.Series'>" for value "adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
revenue                

In [14]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import decimal
import json
import pandas as pd

dynamodb = boto3.resource('dynamodb', region_name='us-west-1', endpoint_url="https://dynamodb.us-west-1.amazonaws.com")
table = dynamodb.Table('movies_metadata')

fe = Key('revenue').gt(500000000)
pe = "id, revenue, info"
# Expression Attribute Names for Projection Expression only.
esk = None

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    Limit=100
    )

dfResults = pd.DataFrame(columns=['id','revenue','title'])
index=0
resultlist=[]
ids=[]
titles=[]
revenues=[]
infos=[]

for i in response['Items']:
    #print(json.dumps(i, cls=DecimalEncoder))
    #print(json.dumps(i['info'], cls=DecimalEncoder))
    info = json.loads(i['info'])
    title = info['original_title']
    myid = int(i['id'])
    revenue = int(i['revenue'])
    ids.append(myid)
    titles.append(title)
    revenues.append(revenue)
    infos.append(info)
    
    #print(myid, revenue, info)
    #record=[myid, title, revenue, info]
    #resultlist.extend((myid, revenue, title, info))
    #resultlist.append(record)
    #dfResults.loc[index, 'id'] = myid
    #dfResults.loc[index, 'revenue'] = revenue
    #dfResults.loc[index, 'title'] = title
    #dfResults.loc[index, 'info'] = info
    index = index + 1

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )

    for i in response['Items']:
        info = json.loads(i['info'])
        title = info['original_title']
        myid = int(i['id'])
        revenue = int(i['revenue'])
        ids.append(myid)
        titles.append(title)
        revenues.append(revenue)
        infos.append(info)
        
        #print(myid, revenue, info)
        #record=[myid, title, revenue, info]
        #resultlist.extend((myid, revenue, info))
        #resultlist.append(record)
        #dfResults.loc[index, 'id'] = myid
        #dfResults.loc[index, 'revenue'] = revenue
        #dfResults.loc[index, 'title'] = title
        #dfResults.loc[index, 'info'] = info
        index=index+1

resultlist=[ids, titles, revenues, infos]
jsonlist=json.dumps(resultlist)
print(jsonlist)

[[251, 10528, 1892, 122, 9502, 8587, 10764, 329, 254, 435, 607, 559, 674, 80321, 953, 38757, 6479, 675, 12444, 51497, 58, 11631, 601, 671, 557, 767, 12092, 672, 950, 604, 120, 72105, 810, 155, 1726, 95, 615, 57800, 285, 12, 812, 12155, 1865, 809, 13, 49013, 280, 10192, 27205, 41154, 1593, 602, 62177, 411, 8355, 217, 597, 10681, 20352, 558, 12445, 49026, 49444, 41513, 36557, 1858, 10193, 955, 693, 19995, 24021, 1893, 35, 585, 24428, 121, 9806, 8960, 745, 50619, 10527, 1891, 1930, 38356, 591, 70160, 22, 2062, 1895, 14160, 1894, 37724, 18239, 10138, 11, 14161, 56292, 673, 8373, 74], ["Ghost", "Sherlock Holmes", "Return of the Jedi", "The Lord of the Rings: The Return of the King", "Kung Fu Panda", "The Lion King", "Quantum of Solace", "Jurassic Park", "King Kong", "The Day After Tomorrow", "Men in Black", "Spider-Man 3", "Harry Potter and the Goblet of Fire", "Madagascar 3: Europe's Most Wanted", "Madagascar", "Tangled", "I Am Legend", "Harry Potter and the Order of the Phoenix", "Harry P

In [16]:
sortedresultlist=sorted(resultlist, key=lambda x: x[2], reverse=False)
print(sortedresultlist)

TypeError: '<' not supported between instances of 'str' and 'int'

In [17]:
print(resultlist)

[[251, 10528, 1892, 122, 9502, 8587, 10764, 329, 254, 435, 607, 559, 674, 80321, 953, 38757, 6479, 675, 12444, 51497, 58, 11631, 601, 671, 557, 767, 12092, 672, 950, 604, 120, 72105, 810, 155, 1726, 95, 615, 57800, 285, 12, 812, 12155, 1865, 809, 13, 49013, 280, 10192, 27205, 41154, 1593, 602, 62177, 411, 8355, 217, 597, 10681, 20352, 558, 12445, 49026, 49444, 41513, 36557, 1858, 10193, 955, 693, 19995, 24021, 1893, 35, 585, 24428, 121, 9806, 8960, 745, 50619, 10527, 1891, 1930, 38356, 591, 70160, 22, 2062, 1895, 14160, 1894, 37724, 18239, 10138, 11, 14161, 56292, 673, 8373, 74], ['Ghost', 'Sherlock Holmes', 'Return of the Jedi', 'The Lord of the Rings: The Return of the King', 'Kung Fu Panda', 'The Lion King', 'Quantum of Solace', 'Jurassic Park', 'King Kong', 'The Day After Tomorrow', 'Men in Black', 'Spider-Man 3', 'Harry Potter and the Goblet of Fire', "Madagascar 3: Europe's Most Wanted", 'Madagascar', 'Tangled', 'I Am Legend', 'Harry Potter and the Order of the Phoenix', 'Harry P

In [94]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import decimal
import json

def lambda_handler(event, context):
    # TODO implement
    dynamodb = boto3.resource('dynamodb', region_name='us-west-1', endpoint_url="https://dynamodb.us-west-1.amazonaws.com")
    table = dynamodb.Table('movies_metadata')

    fe = Key('revenue').gt(500000000)
    kce = Key('revenue').gt(100000)
    ke = Key('id').gt(0)
    pe = "id, revenue, info"
    # Expression Attribute Names for Projection Expression only.
    esk = None

    # Helper class to convert a DynamoDB item to JSON.
    class DecimalEncoder(json.JSONEncoder):
        def default(self, o):
            if isinstance(o, decimal.Decimal):
                if o % 1 > 0:
                   return float(o)
                else:
                    return int(o)
            return super(DecimalEncoder, self).default(o)

    response = table.scan(
        FilterExpression=fe,
        ProjectionExpression=pe,
        Limit=100
       )

    count=0

    for i in response['Items']:
        #print(json.dumps(i, cls=DecimalEncoder))
        #print(json.dumps(i['info'], cls=DecimalEncoder))
        info = json.loads(i['info'])
        print(count, info['title'])
        count=count+1

    while 'LastEvaluatedKey' in response:
        response = table.scan(
            ProjectionExpression=pe,
           FilterExpression=fe,
            ExclusiveStartKey=response['LastEvaluatedKey']
            )

        for i in response['Items']:
            #print(json.dumps(i, cls=DecimalEncoder))
            #print(json.dumps(i['info'], cls=DecimalEncoder))
            info = json.loads(i['info'])
            print(count, info['title'])
            count=count+1
            
    return (response)

response = lambda_handler("x", "y")

0 Ghost
1 Sherlock Holmes
2 Return of the Jedi
3 The Lord of the Rings: The Return of the King
4 Kung Fu Panda
5 The Lion King
6 Quantum of Solace
7 Jurassic Park
8 King Kong
9 The Day After Tomorrow
10 Men in Black
11 Spider-Man 3
12 Harry Potter and the Goblet of Fire
13 Madagascar 3: Europe's Most Wanted
14 Madagascar
15 Tangled
16 I Am Legend
17 Harry Potter and the Order of the Phoenix
18 Harry Potter and the Deathly Hallows: Part 1
19 Fast Five
20 Pirates of the Caribbean: Dead Man's Chest
21 Mamma Mia!
22 E.T. the Extra-Terrestrial
23 Harry Potter and the Philosopher's Stone
24 Spider-Man
25 Harry Potter and the Half-Blood Prince
26 Alice in Wonderland
27 Harry Potter and the Chamber of Secrets
28 Ice Age: The Meltdown
29 The Matrix Reloaded
30 The Lord of the Rings: The Fellowship of the Ring
31 Ted
32 Shrek the Third
33 The Dark Knight
34 Iron Man
35 Armageddon
36 The Passion of the Christ
37 Ice Age: Continental Drift
38 Pirates of the Caribbean: At World's End
39 Finding Nem

In [95]:
print(response)

{'Items': [], 'Count': 0, 'ScannedCount': 321, 'ResponseMetadata': {'RequestId': '01IUSJHB43PDRG1OUPECSI7NSJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 10 Mar 2018 03:46:11 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '41', 'connection': 'keep-alive', 'x-amzn-requestid': '01IUSJHB43PDRG1OUPECSI7NSJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '4005695106'}, 'RetryAttempts': 0}}
